<a href="https://colab.research.google.com/github/monalbhuva/Pytorch/blob/main/Pytorch_RNN_Qa_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import pandas as pd

df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [40]:
# Tokenize

def tokenize(text):
  text = text.lower()
  text = text.replace('?', '')
  text = text.replace("'","")
  return text.split()

In [41]:
# vocab

vocab = {'<UNK>' : 0}

def build_vocab(row):
  tokenize_question = tokenize(row['question'])
  tokenize_answer = tokenize(row['answer'])

  marged_tokens = tokenize_question + tokenize_answer

  for token in marged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)


In [42]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [43]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [44]:
len(vocab)

324

In [45]:
# Convert words to numerical indices

def text_to_indices(text,vocab):

  indexd_text = []

  for token in tokenize(text):
    if token in vocab:
      indexd_text.append(vocab[token])
    else:
      indexd_text.append(vocab['<UNK>'])

  return indexd_text

In [46]:
text_to_indices('What is monal', vocab)

[1, 2, 0]

In [47]:
import torch
from torch.utils.data import Dataset, DataLoader

In [48]:
class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index):
    numarical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numarical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(numarical_question), torch.tensor(numarical_answer)

In [49]:
dataset = QADataset(df,vocab)

In [50]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [51]:
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [52]:
import torch.nn as nn

In [60]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [61]:
learning_rate = 0.001
epochs = 20

In [62]:
model = SimpleRNN(len(vocab))

In [63]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [64]:
from logging import critical
# Trainig loop

for epoch in range(epochs):
  total_loss = 0

  for question,answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss
    loss = criterion(output,answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 520.437493
Epoch: 2, Loss: 448.740519
Epoch: 3, Loss: 371.369168
Epoch: 4, Loss: 312.029455
Epoch: 5, Loss: 260.350814
Epoch: 6, Loss: 212.248709
Epoch: 7, Loss: 168.765978
Epoch: 8, Loss: 130.858077
Epoch: 9, Loss: 99.513939
Epoch: 10, Loss: 75.992352
Epoch: 11, Loss: 57.948495
Epoch: 12, Loss: 45.367178
Epoch: 13, Loss: 35.896788
Epoch: 14, Loss: 28.549193
Epoch: 15, Loss: 23.377859
Epoch: 16, Loss: 19.365603
Epoch: 17, Loss: 16.220671
Epoch: 18, Loss: 13.810606
Epoch: 19, Loss: 11.838169
Epoch: 20, Loss: 10.342151


In [67]:
def predict(model,question,threshold = 0):

  # Conver Question to number
  numarical_question = text_to_indices(question,vocab)

  # tensor
  qustion_tensor = torch.tensor(numarical_question).unsqueeze(0)

  # prediction
  output = model(qustion_tensor)

  # conver logist to probs
  probs = torch.nn.functional.softmax(output,dim=1)

  # find index of max prob
  value,index =  torch.max(probs,dim=1)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [70]:
predict(model, "What is the largest planet in our solar system?")

jupiter
